# Optimize Interface Film Position in Gr/Ni(111) Interface

## 0. Introduction

This notebook demonstrates how to create Gr/Ni(111) interface and optimize the film position.

Following the manuscript:

> **Arjun Dahal, Matthias Batzill**
> "Graphene–nickel interfaces: a review"
> Nanoscale, 6(5), 2548. (2014)
> [DOI: 10.1039/c3nr05279f](https://doi.org/10.1039/c3nr05279f)
 
Recreating interface and shifting the film to the most favorable energy position showed in the image below. Fig 1. b.

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/optimization/optimization_interface_film_xy_position_graphene_nickel/0-figure-from-manuscript.webp" alt="Optimal position of the film" width="400"/>

## 1. Prepare the Environment
### 1.1. Install Packages


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_interface_with_min_strain_zsl.ipynb")


### 1.2. Set Parameters


In [ ]:
# Material selection
SUBSTRATE_NAME = "Nickel"
FILM_NAME = "Graphene"

# Slab parameters
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_TERMINATION_FORMULA = None  # if None, the first termination will be used
FILM_VACUUM = 0.0  # in angstroms

SUBSTRATE_MILLER_INDICES = (1, 1, 1)
SUBSTRATE_THICKNESS = 4  # in atomic layers
SUBSTRATE_TERMINATION_FORMULA = None  # if None, the first termination will be used
SUBSTRATE_VACUUM = 0.0  # in angstroms

# Interface parameters
INTERFACE_DISTANCE = 2.58  # Gap between substrate and film, in Angstrom
INTERFACE_VACUUM = 20.0  # Vacuum over film, in Angstrom

# Whether to convert materials to conventional cells before creating slabs.
USE_CONVENTIONAL_CELL = True

# Maximum area for the superlattice search algorithm (the final interface area will be smaller)
MAX_AREA = 350  # in Angstrom^2
# Additional fine-tuning parameters (increase values to get more strained matches):
MAX_AREA_TOLERANCE = 0.09  # in Angstrom^2
MAX_LENGTH_TOLERANCE = 0.05
MAX_ANGLE_TOLERANCE = 0.02

# Whether to reduce the resulting interface cell to the primitive cell after the interface creation.
REDUCE_RESULT_CELL_TO_PRIMITIVE = True

# Optimization parameters
GRID_SIZE = (20, 20)  # Resolution of the x-y grid
GRID_RANGE_X = (-0.5, 0.5)  # Range to search in x direction
GRID_RANGE_Y = (-0.5, 0.5)  # Range to search in y direction
USE_CARTESIAN = False  # Whether to use Cartesian coordinates

# Visualization parameters
SHOW_3D_LANDSCAPE = False  # Whether to show 3D energy landscape
STRUCTURE_REPETITIONS = [3, 3, 1]  # Repetitions for structure visualization


## 2. Create Interface Material
### 2.1. Load Materials


In [ ]:
from utils.visualize import visualize_materials
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

substrate = Material.create(Materials.get_by_name_first_match(SUBSTRATE_NAME))
film = Material.create(Materials.get_by_name_first_match(FILM_NAME))

# Preview materials
visualize_materials([substrate, film], repetitions=STRUCTURE_REPETITIONS, rotation="0x")


### 2.2. Create Slabs

In [ ]:
from mat3ra.made.tools.build.pristine_structures.two_dimensional.slab import SlabConfiguration, SlabBuilder

substrate_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    number_of_layers=SUBSTRATE_THICKNESS,
    vacuum=0.0,
    termination_top_formula=SUBSTRATE_TERMINATION_FORMULA,
    use_conventional_cell=USE_CONVENTIONAL_CELL
)

film_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=film,
    miller_indices=FILM_MILLER_INDICES,
    number_of_layers=FILM_THICKNESS,
    vacuum=0.0,
    termination_bottom_formula=FILM_TERMINATION_FORMULA, # Termination for the bottom of the film slab
    use_conventional_cell=USE_CONVENTIONAL_CELL
)

substrate_slab = SlabBuilder().get_material(substrate_slab_config)
film_slab = SlabBuilder().get_material(film_slab_config)

# Visualize the created slabs
visualize_materials([{"material": film_slab, "title": "Film Slab"}, {"material": substrate_slab, "title": "Substrate Slab"}],
          repetitions=[3, 3, 1], rotation="-90x")


### 2.3. Create Interface


In [ ]:
from mat3ra.made.tools.helpers import create_interface_zsl_between_slabs as create_zsl_interface_between_slabs

interface_material = create_zsl_interface_between_slabs(
    substrate_slab=substrate_slab,
    film_slab=film_slab,
    gap=INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM,
    match_id=0,
    max_area=MAX_AREA,
    max_area_ratio_tol=MAX_AREA_TOLERANCE,
    max_length_tol=MAX_LENGTH_TOLERANCE,
    max_angle_tol=MAX_ANGLE_TOLERANCE,
    reduce_result_cell_to_primitive=REDUCE_RESULT_CELL_TO_PRIMITIVE,
)

interface_material.name = f"{FILM_NAME}_{SUBSTRATE_NAME}_interface"

# Visualize interface
visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS)
visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS, rotation="-90x")


## 3. Optimize Interface Position
### 3.1. Calculate Energy Landscape


In [ ]:
from mat3ra.made.tools.helpers import get_optimal_film_displacement
from mat3ra.made.tools.modify import interface_displace_part
from mat3ra.made.tools.optimize import evaluate_calculator_on_xy_grid
from mat3ra.made.tools.calculate.calculators import InterfaceMaterialCalculator

calculator = InterfaceMaterialCalculator()

# Calculate energy landscape
xy_matrix, energy_matrix = evaluate_calculator_on_xy_grid(
    material=interface_material,
    calculator_function=calculator.get_energy,
    modifier=interface_displace_part,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)

# Find optimal position
optimal_displacement = get_optimal_film_displacement(
    material=interface_material,
    calculator=calculator,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)
print(f"\nOptimal displacement vector: {optimal_displacement}")


### 3.2. Visualize Energy Landscape


In [ ]:
from mat3ra.utils.jupyterlite.plot import plot_2d_heatmap, plot_3d_surface

x_values, y_values = xy_matrix
# Plot energy landscape
plot_2d_heatmap(x_values, y_values, energy_matrix,
                optimal_point=optimal_displacement[:2],
                title="Energy Heatmap",
                labels={"x": "x", "y": "y", "z": "Energy"})
if SHOW_3D_LANDSCAPE:
    plot_3d_surface(x_values, y_values, energy_matrix,
                    optimal_point=optimal_displacement[:2],
                    title="Energy Landscape",
                    labels={"x": "x", "y": "y", "z": "Energy"})

# Create and visualize optimized material
optimized_material = interface_displace_part(
    interface_material,
    displacement=optimal_displacement,
    use_cartesian_coordinates=USE_CARTESIAN
)

### 3.3. Visualize Material


In [ ]:
print("\nVisualization of original and optimized materials:")
visualize_materials([{"material": interface_material, "title": "Original Interface"},
                     {"material": optimized_material, "title": "Optimized Interface"}],
                    repetitions=STRUCTURE_REPETITIONS,
                    )
visualize_materials([{"material": interface_material, "title": "Original Interface"},
                     {"material": optimized_material, "title": "Optimized Interface"}],
                    repetitions=STRUCTURE_REPETITIONS,
                    rotation="-90x")

# 4. Save optimized material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

optimized_material.name = f"{interface_material.name}_optimized_xy"
set_materials(optimized_material)
download_content_to_file(optimized_material.to_json(), f"{interface_material.name}_optimized_xy.json")